In [19]:
import numpy as np
import pandas as pd
def Linear_Max_Min(attribute, Type):
    """
    Patro, S. and Sahu, K.K., 2015. Normalization: A preprocessing stage.
    arXiv preprint arXiv:1503.06462.
    """
    MIN = min(attribute)
    MAX = max(attribute)
    Var = MAX - MIN
    attribute = np.array(attribute)
    if Type == "profit":
        Attribute = (attribute - MIN)/Var
    elif Type == "cost":
        Attribute = (MAX - attribute)/Var
    
    return Attribute

def Linear_Max(attribute, Type):
    """
    Çelen, A., 2014. Comparative analysis of normalization procedures in
    TOPSIS method: with an application to Turkish deposit banking market.
    Informatica, 25(2), pp.185-208.
    """
    MAX = max(attribute)
    attribute = np.array(attribute)
    if Type == "profit":
        Attribute = attribute/MAX
    elif Type == "cost":
        Attribute = 1 - attribute/MAX
    
    return Attribute

def Linear_sum(attribute, Type):
    """
    Jahan, A. and Edwards, K.L. (2014) ‘A state-of-the-art survey on the
    influence of normalization techniques in ranking: improving the materials
    selection process in engineering design’, Mater. Des., Vol. 65, No. 2015,
    pp.335–342.
    """
    SUM_1 = sum(attribute)
    SUM_2 = sum(1/attribute)
    attribute = np.array(attribute)
    if Type == "profit":
        Attribute = attribute/SUM_1 
    elif Type == "cost":
        Attribute = (1/attribute)/SUM_2
        
    return Attribute


def Vector_Normalization(attribute, Type):
    """
    Jahan, A. and Edwards, K.L. (2014) ‘A state-of-the-art survey on the
    influence of normalization techniques in ranking: improving the materials
    selection process in engineering design’, Mater. Des., Vol. 65, No. 2015,
    pp.335–342.
    """
    r = (sum(attribute**2))**(0.5)
    if Type == "profit":
        Attribute = attribute/r 
    elif Type == "cost":
        Attribute = 1 - attribute/r 
        
    return Attribute


def Logarithmic_normalisation(attribute, Type):
    """
    Jahan, A. and Edwards, K.L. (2014) ‘A state-of-the-art survey on the
    influence of normalization techniques in ranking: improving the materials
    selection process in engineering design’, Mater. Des., Vol. 65, No. 2015,
    pp.335–342.
    """
    r = np.log(np.prod(attribute))
    if Type == "profit":
        Attribute = np.log(attribute)/r
        
    if Type == "cost":
        Attribute = (1 - (np.log(attribute)/r))/(len(attribute)-1)
        
    return Attribute


In [20]:
from Normalization import *

def Normalize_Decision_Matrix(Decision_Matrix, Attribute_Type, Normal_Method):
    for i in range(0, len(Attribute_Type)):
        if Attribute_Type[i] == 1:
            Type = "profit"
        elif Attribute_Type[i] == 0:
            Type = "cost"
        else:
            "Please, fill the correct attribute type"

        Decision_Matrix.iloc[:, i] = Normal_Method(Decision_Matrix.iloc[:, i], Type)
    return Decision_Matrix


In [21]:
def Weight_Prod(Decision_Matrix, weights):
    if len(weights) != Decision_Matrix.shape[1]:
        raise ValueError("The number of weights does not match the number of criteria.")
    
    normalized_weights = [float(w)/sum(weights) for w in weights]
    
    for i in range(Decision_Matrix.shape[1]):
        Decision_Matrix.iloc[:, i] = Decision_Matrix.iloc[:, i] * normalized_weights[i]
    
    return Decision_Matrix

In [22]:
from Weighted_Normalized_Decision_Matrix import *

def Ideal_Solution(Weighted_normalizaed_Decision_matrix, Attribute_Type):
    Positive_Ideal_Solutions = []
    Negative_Ideal_Solutions = []
    for i in range(0, len(Attribute_Type)):
        if Attribute_Type[i] == 1:
            Positive_Ideal = max(Weighted_normalizaed_Decision_matrix.iloc[:, i]) 
            Negative_Ideal = min(Weighted_normalizaed_Decision_matrix.iloc[:, i])
            
            Positive_Ideal_Solutions.append(Positive_Ideal)
            Negative_Ideal_Solutions.append(Negative_Ideal)
        elif Attribute_Type[i] == 0:
            Positive_Ideal = min(Weighted_normalizaed_Decision_matrix.iloc[:, i]) 
            Negative_Ideal = max(Weighted_normalizaed_Decision_matrix.iloc[:, i])
        
            Positive_Ideal_Solutions.append(Positive_Ideal)
            Negative_Ideal_Solutions.append(Negative_Ideal)
            
    return (Positive_Ideal_Solutions, Negative_Ideal_Solutions)

In [23]:
from Ideal_Solution import *
import numpy as np
import pandas as pd
def Euclidean_Distance(Ideal_Points, Weighted_normalizaed_Decision_matrix, Attribute_Type):
    Positive_Ideal_Point = Ideal_Solution(Weighted_normalizaed_Decision_matrix, Attribute_Type)[0]
    Negative_Ideal_Point = Ideal_Solution(Weighted_normalizaed_Decision_matrix, Attribute_Type)[1]
    
    Negative_Si = (np.sum((Weighted_normalizaed_Decision_matrix-Negative_Ideal_Point)**2, axis = 1))**0.5  
    Positive_Si = np.sum((Weighted_normalizaed_Decision_matrix-Positive_Ideal_Point)**2, axis = 1)**0.5

    C = Negative_Si/(Negative_Si + Positive_Si)
    return (Weighted_normalizaed_Decision_matrix,C)

In [24]:
from topsis import topsis
import pandas as pd
import numpy as np
import mcdm

from Normalization import *
from Normalized_Decision_Matrix import *
from Ideal_Solution import Ideal_Solution
from Distance_Between_Ideal_and_Alternatives import *

# path = "..."
# name= "Sample.xlsx"

# Data = pd.read_excel(path + name)
name = "DataKandidatSekJur.xlsx"
Data = pd.read_excel(name)
Data.index = Data['Candidate List']
Data = Data.iloc[:, 1:]

num_criteria = Data.shape[1]
weights = [1/num_criteria] * num_criteria

PC = [1] * num_criteria

def TOPSIS(Decision_Matrix, Weights, Attribute_Type, Normalization_Method = Linear_sum):
    
    """
    TOPSIS implementation based on this publication:
        * C.-L. Hwang and K. Yoon, Multiple attribute decision making,
        ser. Lecture Notes in Economics and Mathematical Systems.
        Springer-Verlag Berlin Heidelberg, 1981, vol. 186,ethod = Normalization_Method
        ISBN: 9783540105589.
        Book Link: https://www.springer.com/gp/book/9783540105589
    """

    if len(Weights) != Decision_Matrix.shape[1]:
        raise ValueError("The number of weights must match the number of criteria.")
    if len(Attribute_Type) != Decision_Matrix.shape[1]:
        raise ValueError("The number of attribute types must match the number of criteria.")
    
    Normalizaed_Decision_matrix = Normalize_Decision_Matrix(Decision_Matrix = Decision_Matrix,
                                                            Attribute_Type=Attribute_Type,
                                                            Normal_Method=Normalization_Method)
     
    Weighted_Normalizaed_Decision_matrix = Weight_Prod(Decision_Matrix = Normalizaed_Decision_matrix, weights = Weights)
    
    Ideal_Solutions = Ideal_Solution(Weighted_Normalizaed_Decision_matrix, Attribute_Type)
    a = Euclidean_Distance(Ideal_Solutions, Weighted_Normalizaed_Decision_matrix, Attribute_Type)
    return a
 
Output = TOPSIS(Data, weights, PC,Normalization_Method = Linear_Max_Min)

Scores=pd.DataFrame(Output[1])
Normalized_Weighted_Decision_Matrix = Output[0]
df=pd.concat([Normalized_Weighted_Decision_Matrix, Scores], axis=1)

print("Normalized Weighted Decision Matrix:\n", Output[0])
pd.set_option('display.max_rows', None)
print("Scores:\n", Output[1])
for index, score in Output[1].items():
    print(f"{index}: {score}")


Normalized Weighted Decision Matrix:
                      PF1       PF2       PF3       PF4       PF5       PF6  \
Candidate List                                                               
Candidate A     0.026786  0.035714  0.035714  0.035714  0.017857  0.026786   
Candidate B     0.035714  0.026786  0.011905  0.000000  0.000000  0.026786   
Candidate C     0.017857  0.008929  0.023810  0.035714  0.035714  0.035714   
Candidate D     0.035714  0.017857  0.000000  0.017857  0.035714  0.008929   
Candidate E     0.000000  0.000000  0.011905  0.000000  0.017857  0.000000   
Candiate F      0.035714  0.017857  0.000000  0.017857  0.035714  0.008929   

                     PF7       PF8       PF9        K1  ...        S1  \
Candidate List                                          ...             
Candidate A     0.035714  0.000000  0.035714  0.024855  ...  0.011905   
Candidate B     0.035714  0.035714  0.035714  0.000000  ...  0.000000   
Candidate C     0.000000  0.000000  0.000000 